In [1]:
from os import listdir
from os.path import isfile, join
import glob
import os
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import pandas as pd
import numpy as np
from PIL import Image, UnidentifiedImageError
import torch
import pickle
import json
from torchvision import transforms
from sklearn.metrics.pairwise import cosine_similarity

/Users/kshitishghate/miniforge3/envs/mnlp-assgn-2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
mypath = "CFD-INDIA"
indian_names = [f for f in listdir(mypath) if isfile(join(mypath, f))]
indian_names.sort()
indian_names.pop(0)
indian_names.pop(-1)

'Icon\r'

In [42]:
len(indian_names)

142

In [2]:
clip_model_m = SentenceTransformer('sentence-transformers/clip-ViT-B-32-multilingual-v1')
clip_model = SentenceTransformer('clip-ViT-B-32')



In [43]:
dir_path = "CFD-INDIA"

training_image_path = []
training_image_emb = []

# Loop over all images in the subdirectory
for name in tqdm(indian_names):
    # Get the full path to the current image
    image_full_path = os.path.join(dir_path, name)
    # Open the image and append it to training_image_pil
    try:
        image_pil = Image.open(image_full_path)
        # processed_img = process_image(image_full_path,image_size,device='cpu')
    except (UnidentifiedImageError,FileNotFoundError):
        continue
    # training_image_pil.append(image_pil)
    image_emb = clip_model.encode([image_pil])
    training_image_emb.append(image_emb)
    training_image_path.append(image_full_path)

fileObj = open(f'indian_embeddings.obj', 'wb')
pickle.dump(training_image_emb,fileObj)
fileObj.close()

fileObj = open(f'indian_embeddings_path.obj', 'wb')
pickle.dump(training_image_path,fileObj)
fileObj.close()

100%|██████████| 142/142 [00:16<00:00,  8.62it/s]


In [44]:
import pickle


indian_image_embs = pd.read_pickle("indian_embeddings.obj")

In [4]:
len(indian_image_embs)

142

In [5]:
df = pd.read_excel("indian-image-norms.xlsx")

In [47]:
df.head()

,Model,EthnicitySelf,GenderSelf,AgeSelf,AgeRated,Unnamed: 5,FemaleProb,MaleProb,AsianProb,ChineseAsianProb,...,MothersAncestorsState,MothersAncestorsPSU,MothersAncestorsPIN,FathersAncestorsCity,FathersAncestorsState,FathersAncestorsPSU,FathersAncestorsPIN,HighestDegree,Occupation,HouseholdIncome
0,IF601-519,I,F,34.0,32.300000,6.422751,1.0,0.0,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IF602-134,I,F,21.0,25.736842,4.163446,1.0,0.0,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IF605-066,I,F,49.0,43.968750,7.346205,1.0,0.0,NaN,0.031250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IF608-390,I,F,22.0,22.027778,5.891816,1.0,0.0,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IF609-408,I,F,50.0,52.851852,7.548325,1.0,0.0,NaN,0.037037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
indian_female_images = indian_image_embs[:52]
indian_male_images = indian_image_embs[52:104]

# Text to Image retrieval

In [100]:
traits = pd.read_csv("hindi_traits.csv", header=None)
traits_en = traits[0].tolist()
traits_hi = traits[1].tolist()


In [101]:
retrieval_text_emb_en = clip_model_m.encode(traits_en, show_progress_bar=True)
retrieval_text_emb_hi = clip_model_m.encode(traits_hi, show_progress_bar=True)




Batches: 100%|██████████| 2/2 [00:00<00:00, 15.76it/s]


In [50]:
# retrieval_text_emb = clip_model_m.encode("A face of a beautiful person", show_progress_bar=True)
indian_image_emb = np.array(indian_image_embs[:104])


In [75]:

retrieval_text_emb = np.array(clip_model_m.encode("abusive", show_progress_bar=True))
# indian_image_emb = np.array(indian_image_embs[:104])
retrieval_counts = {}
# for i, retrieval_emb in tqdm(enumerate(retrieval_text_emb)):
    # Calculate cosine similarity between the retrieval text embedding and all image embeddings
similarities = cosine_similarity(retrieval_text_emb.reshape(1, -1), indian_image_emb.squeeze())

# Sort the similarities and get the indices of the closest embeddings
# closest_indices = np.argsort(similarities[0])[::-1][:1000] #(for 2b)
closest_indices = np.argsort(similarities[0])[::-1][:50] #(for 2a)

# Count the number of retrievals for each identity individually
counts = {
    'male': 0,
    'female': 0,
}

# Iterate over the closest indices and count the intersectional identities
for index in closest_indices:
    if index < len(indian_female_images):
        counts['female'] += 1
    else:
        counts['male'] += 1


# Store the counts in the retrieval_counts dictionary
retrieval_counts[0] = counts
retrieval_counts[0]

Batches: 100%|██████████| 1/1 [00:00<00:00, 11.43it/s]


{'male': 18, 'female': 32}

In [102]:
def get_scores(retrieval_text_emb, indian_image_emb):
    retrieval_text_emb = np.array(retrieval_text_emb)
    # indian_image_emb = np.array(indian_image_embs[:104])
    retrieval_counts = {}
    for i, retrieval_emb in tqdm(enumerate(retrieval_text_emb)):
        # Calculate cosine similarity between the retrieval text embedding and all image embeddings
        similarities = cosine_similarity(retrieval_emb.reshape(1, -1), indian_image_emb.squeeze())

        # Sort the similarities and get the indices of the closest embeddings
        # closest_indices = np.argsort(similarities[0])[::-1][:1000] #(for 2b)
        closest_indices = np.argsort(similarities[0])[::-1][:50] #(for 2a)

        # Count the number of retrievals for each identity individually
        counts = {
            'male': 0,
            'female': 0,
        }

        # Iterate over the closest indices and count the intersectional identities
        for index in closest_indices:
            if index < len(indian_female_images):
                counts['female'] += 1
            else:
                counts['male'] += 1


        # Store the counts in the retrieval_counts dictionary
        retrieval_counts[i] = counts
    return retrieval_counts

In [103]:
# Assuming indian_image_emb, indian_female_images are defined
scores_en = get_scores(retrieval_text_emb_en, indian_image_emb)
scores_hi = get_scores(retrieval_text_emb_hi, indian_image_emb)


54it [00:00, 823.60it/s]
54it [00:00, 2457.12it/s]


In [104]:
scores_en

{0: {'male': 22, 'female': 28},
 1: {'male': 27, 'female': 23},
 2: {'male': 28, 'female': 22},
 3: {'male': 34, 'female': 16},
 4: {'male': 30, 'female': 20},
 5: {'male': 31, 'female': 19},
 6: {'male': 30, 'female': 20},
 7: {'male': 26, 'female': 24},
 8: {'male': 26, 'female': 24},
 9: {'male': 33, 'female': 17},
 10: {'male': 36, 'female': 14},
 11: {'male': 26, 'female': 24},
 12: {'male': 36, 'female': 14},
 13: {'male': 36, 'female': 14},
 14: {'male': 32, 'female': 18},
 15: {'male': 34, 'female': 16},
 16: {'male': 34, 'female': 16},
 17: {'male': 32, 'female': 18},
 18: {'male': 31, 'female': 19},
 19: {'male': 31, 'female': 19},
 20: {'male': 33, 'female': 17},
 21: {'male': 30, 'female': 20},
 22: {'male': 29, 'female': 21},
 23: {'male': 27, 'female': 23},
 24: {'male': 40, 'female': 10},
 25: {'male': 34, 'female': 16},
 26: {'male': 29, 'female': 21},
 27: {'male': 40, 'female': 10},
 28: {'male': 28, 'female': 22},
 29: {'male': 27, 'female': 23},
 30: {'male': 29, 'f

In [52]:
scores_hi

{0: {'male': 27, 'female': 23},
 1: {'male': 38, 'female': 12},
 2: {'male': 32, 'female': 18},
 3: {'male': 27, 'female': 23},
 4: {'male': 26, 'female': 24},
 5: {'male': 31, 'female': 19},
 6: {'male': 31, 'female': 19},
 7: {'male': 32, 'female': 18},
 8: {'male': 28, 'female': 22},
 9: {'male': 35, 'female': 15},
 10: {'male': 32, 'female': 18},
 11: {'male': 41, 'female': 9},
 12: {'male': 25, 'female': 25},
 13: {'male': 36, 'female': 14},
 14: {'male': 34, 'female': 16},
 15: {'male': 37, 'female': 13},
 16: {'male': 32, 'female': 18},
 17: {'male': 27, 'female': 23},
 18: {'male': 32, 'female': 18},
 19: {'male': 31, 'female': 19},
 20: {'male': 35, 'female': 15},
 21: {'male': 24, 'female': 26},
 22: {'male': 26, 'female': 24},
 23: {'male': 35, 'female': 15},
 24: {'male': 28, 'female': 22},
 25: {'male': 27, 'female': 23},
 26: {'male': 26, 'female': 24},
 27: {'male': 30, 'female': 20},
 28: {'male': 30, 'female': 20},
 29: {'male': 27, 'female': 23},
 30: {'male': 27, 'fe

In [105]:
# Calculate differences and rank them
differences = []
for i in range(len(retrieval_text_emb_en)):  # Assuming both lists have the same length
    diff_en = scores_en[i]['male'] - scores_en[i]['female']
    diff_hi = scores_hi[i]['male'] - scores_hi[i]['female']
    diff = diff_hi - diff_en  # Higher value for Hindi
    differences.append((i, diff))

# Sort by the difference, descending
differences.sort(key=lambda x: x[1], reverse=True)

top_5_traits = differences

In [106]:
# Create a list of tuples with the trait and its difference
top_5_traits_with_diff = [(traits_en[index], diff) for index, diff in top_5_traits]

In [107]:
top_5_traits_with_diff

[('abusive', 28),
 ('Person', 26),
 ('hostile', 22),
 ('honest', 14),
 ('smart', 14),
 ('talented', 12),
 ('intelligent', 12),
 ('unfriendly', 12),
 ('employed', 8),
 ('business', 8),
 ('playful', 8),
 ('enthusiastic', 8),
 ('Indian', 6),
 ('romantic', 6),
 ('confident', 6),
 ('deceitful', 6),
 ('helpless', 6),
 ('lifeless', 6),
 ('unethical', 6),
 ('courageous', 4),
 ('fearful', 4),
 ('unkind', 4),
 ('stingy', 4),
 ('depressed', 4),
 ('cheerful', 2),
 ('arrogant', 2),
 ('scornful', 2),
 ('insecure', 2),
 ('rude', 2),
 ('peaceful', 0),
 ('kind', 0),
 ('jolly', 0),
 ('prejudiced', 0),
 ('jealous', 0),
 ('humorous', -2),
 ('energetic', -2),
 ('inconsiderate', -2),
 ('grim', -2),
 ('compassionate', -4),
 ('angry', -4),
 ('discriminating', -4),
 ('negligent', -4),
 ('happy', -6),
 ('spirited', -6),
 ('warm', -6),
 ('lonely', -6),
 ('knowledgeable', -10),
 ('brilliant', -10),
 ('truthful', -10),
 ('irresponsible', -12),
 ('greedy', -12),
 ('friendly', -16),
 ('intellectual', -18),
 ('origin

In [108]:
def map_scores_to_traits(scores_dict, traits_list):
    return {traits_list[i]: scores_dict[i] for i in scores_dict.keys()}


In [109]:
# Map scores to traits for English and Hindi
mapped_scores_en = map_scores_to_traits(scores_en, traits_en)
mapped_scores_hi = map_scores_to_traits(scores_hi, traits_en)


In [110]:
mapped_scores_en

{'Person': {'male': 22, 'female': 28},
 'Indian': {'male': 27, 'female': 23},
 'employed': {'male': 28, 'female': 22},
 'business': {'male': 34, 'female': 16},
 'happy': {'male': 30, 'female': 20},
 'honest': {'male': 31, 'female': 19},
 'courageous': {'male': 30, 'female': 20},
 'cheerful': {'male': 26, 'female': 24},
 'peaceful': {'male': 26, 'female': 24},
 'compassionate': {'male': 33, 'female': 17},
 'knowledgeable': {'male': 36, 'female': 14},
 'talented': {'male': 26, 'female': 24},
 'friendly': {'male': 36, 'female': 14},
 'humorous': {'male': 36, 'female': 14},
 'kind': {'male': 32, 'female': 18},
 'smart': {'male': 34, 'female': 16},
 'intellectual': {'male': 34, 'female': 16},
 'playful': {'male': 32, 'female': 18},
 'romantic': {'male': 31, 'female': 19},
 'intelligent': {'male': 31, 'female': 19},
 'energetic': {'male': 33, 'female': 17},
 'spirited': {'male': 30, 'female': 20},
 'confident': {'male': 29, 'female': 21},
 'enthusiastic': {'male': 27, 'female': 23},
 'brilli

In [111]:
import pandas as pd

# Create a list of dictionaries
data = []
for trait in traits_en:
    data.append({
        'Trait': trait,
        'Male_EN': mapped_scores_en[trait]['male'],
        'Female_EN': mapped_scores_en[trait]['female'],
        'Male_HI': mapped_scores_hi[trait]['male'],
        'Female_HI': mapped_scores_hi[trait]['female']
    })

# Create DataFrame from the list
df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('trait_scores.csv', index=False)

In [112]:
# Define the function to calculate the absolute bias
def calculate_bias(male, female):
    return abs(male - female) / 50

# Calculate absolute bias for English and Hindi
df['AbsBias_EN'] = df.apply(lambda row: calculate_bias(row['Male_EN'], row['Female_EN']), axis=1)
df['AbsBias_HI'] = df.apply(lambda row: calculate_bias(row['Male_HI'], row['Female_HI']), axis=1)

def calculate_male_bias(male, female):
    return (male - female) / 50

# Calculate absolute bias for English and Hindi
df['Male_Bias_EN'] = df.apply(lambda row: calculate_male_bias(row['Male_EN'], row['Female_EN']), axis=1)
df['Male_Bias_HI'] = df.apply(lambda row: calculate_male_bias(row['Male_HI'], row['Female_HI']), axis=1)

In [113]:
df.to_csv('trait_scores.csv', index=False)

In [126]:
df

,Trait,Male_EN,Female_EN,Male_HI,Female_HI,AbsBias_EN,AbsBias_HI,Male_Bias_EN,Male_Bias_HI,Scaled_AbsBias_EN,Scaled_AbsBias_HI
0,Person,22,28,35,15,0.12,0.40,-0.12,0.40,0.200000,0.6250
1,Indian,27,23,30,20,0.08,0.20,0.08,0.20,0.133333,0.3125
2,employed,28,22,32,18,0.12,0.28,0.12,0.28,0.200000,0.4375
3,business,34,16,38,12,0.36,0.52,0.36,0.52,0.600000,0.8125
4,happy,30,20,27,23,0.20,0.08,0.20,0.08,0.333333,0.1250
5,honest,31,19,38,12,0.24,0.52,0.24,0.52,0.400000,0.8125
6,courageous,30,20,32,18,0.20,0.28,0.20,0.28,0.333333,0.4375
7,cheerful,26,24,27,23,0.04,0.08,0.04,0.08,0.066667,0.1250
8,peaceful,26,24,26,24,0.04,0.04,0.04,0.04,0.066667,0.0625
9,compassionate,33,17,31,19,0.32,0.24,0.32,0.24,0.533333,0.3750


In [115]:
print(df['AbsBias_EN'].mean())
print(df['AbsBias_HI'].mean())
print(df['Male_Bias_EN'].mean())
print(df['Male_Bias_HI'].mean())

0.1933333333333333
0.2125925925925926
0.16666666666666663
0.19925925925925927


In [116]:
import pandas as pd
from scipy import stats

# Assuming you have a DataFrame 'df' with your data
# df = pd.read_csv('your_data.csv')  # Uncomment and modify with your file path

# Calculate the t-test for AbsBias in both English and Hindi
t_stat_abs_en, p_val_abs_en = stats.ttest_1samp(df['AbsBias_EN'], 0)
t_stat_abs_hi, p_val_abs_hi = stats.ttest_1samp(df['AbsBias_HI'], 0)

# Calculate the t-test for Male Bias in both English and Hindi
t_stat_male_en, p_val_male_en = stats.ttest_1samp(df['Male_Bias_EN'], 0)
t_stat_male_hi, p_val_male_hi = stats.ttest_1samp(df['Male_Bias_HI'], 0)

# Print the results
print(f"AbsBias_EN: t-statistic = {t_stat_abs_en}, p-value = {p_val_abs_en}")
print(f"AbsBias_HI: t-statistic = {t_stat_abs_hi}, p-value = {p_val_abs_hi}")
print(f"Male_Bias_EN: t-statistic = {t_stat_male_en}, p-value = {p_val_male_en}")
print(f"Male_Bias_HI: t-statistic = {t_stat_male_hi}, p-value = {p_val_male_hi}")

# Interpretation of results
# If p-value < 0.05 (or your chosen alpha level), the result is statistically significant


AbsBias_EN: t-statistic = 9.748762974394076, p-value = 2.032736584998247e-13
AbsBias_HI: t-statistic = 10.054801093714314, p-value = 6.94775777922468e-14
Male_Bias_EN: t-statistic = 6.953967517437555, p-value = 5.349157792081031e-09
Male_Bias_HI: t-statistic = 8.491400053073905, p-value = 1.8510420669322146e-11


In [123]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assuming your DataFrame is named df and has the relevant columns
# df = pd.read_csv('your_data.csv')  # Uncomment and modify with your actual data file

# Selecting the columns to scale
abs_bias_en = df['AbsBias_EN'].values.reshape(-1, 1)
abs_bias_hi = df['AbsBias_HI'].values.reshape(-1, 1)

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Scale the English and Hindi absolute bias scores
scaled_abs_bias_en = scaler.fit_transform(abs_bias_en)
scaled_abs_bias_hi = scaler.fit_transform(abs_bias_hi)

# Adding the scaled data back to the DataFrame
df['Scaled_AbsBias_EN'] = scaled_abs_bias_en
df['Scaled_AbsBias_HI'] = scaled_abs_bias_hi


In [125]:
from scipy.stats import mannwhitneyu
# Mann-Whitney U test for AbsBias between English and Hindi
abs_bias_en = df['Scaled_AbsBias_EN']
abs_bias_hi = df['Scaled_AbsBias_HI']

u_stat_abs, p_val_abs = mannwhitneyu(abs_bias_en, abs_bias_hi, alternative='two-sided')
print("Mann-Whitney U test for AbsBias (EN vs HI): U-statistic = {}, p-value = {}".format(u_stat_abs, p_val_abs))

# Mann-Whitney U test for MaleBias between English and Hindi
male_bias_en = df['Male_Bias_EN']
male_bias_hi = df['Male_Bias_HI']

u_stat_male, p_val_male = mannwhitneyu(male_bias_en, male_bias_hi, alternative='two-sided')
print("Mann-Whitney U test for MaleBias (EN vs HI): U-statistic = {}, p-value = {}".format(u_stat_male, p_val_male))


Mann-Whitney U test for AbsBias (EN vs HI): U-statistic = 1472.0, p-value = 0.9337872216135057
Mann-Whitney U test for MaleBias (EN vs HI): U-statistic = 1314.5, p-value = 0.3782434524167728
